# Calculate correlations between variables in the data

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from holoviews import opts
from holoviews.plotting.links import DataLink
hv.extension('bokeh')
from bokeh.resources import INLINE

import functions_bondjango as bd
import functions_kinematic as fk
import functions_preprocessing as fp
import functions_misc as fm
import pandas as pd
import numpy as np
import h5py

import pixiedust

from IPython.core.debugger import set_trace

In [ ]:
# get the data paths
try: 
    data_path = snakemake.input[0]
except NameError:
    # define the search string
    search_string = 'result:succ, lighting:normal, rig:miniscope, =analysis_type:aggFullCA'
    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = data_all[0]['analysis_path']
print(data_path)

# load the data
# with pd.HDFStore(data_all[0]['analysis_path']) as h:
with pd.HDFStore(data_path) as h:

    # get a list of the existing keys
    keys = h.keys()
    # if it's only one key, just load the file
    if len(keys) == 1:
        data = h[keys[0]]
    else:   
        # allocate a dictionary for them
        data = {}
        # extract the animals present
        animal_list = [el.split('/')[1] for el in keys]
        # get the unique animals
        unique_animals = np.unique(animal_list)
        # for all the animals
        for animal in unique_animals:
            # allocate a dictionary for the tables
            sub_dict = {}
            # get the unique dates for this animal
            date_list = [el.split('/')[2] for el in keys if animal in el]
            # for all the unique dates
            for date in date_list:
                # get the corresponding key
                current_key = [el for el in keys if (animal in el) and (date in el)][0]
                print(current_key)
                # load the table into the dictionary (minus the d at the beginning, added cause natural python naming)
                sub_dict[date[1:]] = h[current_key]
            # save the dictionary into the corresponding entry of the animal dictionary
            data[animal] = sub_dict
        


In [ ]:
# Plot the average cell activity
# allocate a dict for the holomap
holo_dict = {}
average_dict = {}


# check if dictionary or dataframe
if isinstance(data, dict):
    # get the dates for this mouse
    mice = data.keys()
else:
    mice = ['All']
# for all the mice
for mouse in mice:
    sub_dict = {}
    # check if dictionary or dataframe
    if isinstance(data, dict):
        # get the dates for this mouse
        dates = data[mouse].keys()
    else:
        dates = ['All']
    # for all the dates
    for day in dates:
        # get the table
        if isinstance(data, dict):
            sub_data = data[mouse][day]
        else:
            sub_data = data
        # get the available columns
        labels = list(sub_data.columns)
        cells = [el for el in labels if 'cell' in el]
        not_cells = [el for el in labels if 'cell' not in el]
        # calculate the average neuronal response
        calcium_data = sub_data.loc[:, cells].to_numpy()
        calcium_data = np.nanmean(calcium_data, axis=1)
        
        # select a subset of columns
#         sub_data = sub_data[cells+['mouse_speed']].copy()
        sub_data.loc[sub_data['mouse_speed']>2000, 'mouse_speed'] = 0

#         sub_data = sub_data.drop(columns=cells + ['frame', 'time_vector'])
#         sub_data['calcium_average'] = calcium_data
        labels = sub_data.columns
        # save the array for computations
        sub_dict[day] = sub_data
        # get the array
        sub_array = sub_data.to_numpy()
        sub_array = fm.normalize_matrix(sub_array, axis=0)
        # create the labels
        y_labels = [(idx+0.5, el) for idx, el in enumerate(labels)]

#         raw_image = hv.Image(sub_array, bounds=[0,0,len(y_labels),10], label=mouse+'_'+day)
        raw_image = hv.Image(sub_array, label=mouse+'_'+day)
        raw_image.opts(width=800, height=600, invert_axes=True, invert_yaxis=True, 
                       invert_xaxis=True, cmap='Viridis', yticks=y_labels, tools=['hover'], shared_axes=False)

        holo_dict[(mouse, day)] = raw_image
    # save the sub array for computations
    average_dict[mouse] = sub_dict
# holo_image = hv.Layout(plot_list, group='images').cols(1)

holo_image = pn.panel(hv.HoloMap(holo_dict, kdims=['mouse', 'day']), center=True, widget_location='top')
# holo_image = hv.GridSpace(holo_dict, kdims=['mouse', 'day']).opts(plot_size=300)
holo_image

In [ ]:
# Calculate the correlation matrices and display
# allocate a dict for the holomap
holo_dict = {}

# check if dictionary or dataframe
if isinstance(average_dict, dict):
    # get the dates for this mouse
    mice = average_dict.keys()
else:
    mice = ['All']
# for all the mice
for mouse in mice:
    # check if dictionary or dataframe
    if isinstance(average_dict, dict):
        # get the dates for this mouse
        dates = average_dict[mouse].keys()
    else:
        dates = ['All']
    # allocate memory for the average
    average_perday = []
    # for all the dates
    for day in dates:
        # get the table
        if isinstance(average_dict, dict):
            sub_data = average_dict[mouse][day].copy()
        else:
            sub_data = average_dict
            
#         # replace speed with a binary vector
#         sub_data['mouse_speed'] = (sub_data['mouse_speed']>200).astype(float)
        # get the available columns
        labels = list(sub_data.columns)
        # get the correlation
        sub_array = np.corrcoef(sub_data.to_numpy().T)

        # store the matrix to take the average
        average_perday.append(sub_array)
    
        # create the labels
        y_labels = [(idx+0.5, el) for idx, el in enumerate(labels)]
        x_labels = [(len(labels)-idx-0.5, el) for idx, el in enumerate(labels)]

        raw_image = hv.Image(sub_array, bounds=[0,0,len(y_labels),len(x_labels)], label=mouse+'_'+day).redim.range(z=(-1, 1))
        raw_image.opts(width=800, height=600, invert_axes=True, invert_yaxis=True, 
                       invert_xaxis=True, cmap='Viridis', yticks=y_labels, xticks=x_labels,
                       tools=['hover'], shared_axes=False, xrotation=45, aspect='equal')

        holo_dict[(mouse, day)] = raw_image
#         holo_dict[(mouse, day)] = hv.Curve((np.arange(sub_data['mouse_speed'].shape[0]), sub_data['mouse_speed']))

#     # add the average for plotting
#     sub_array = np.nanmean(np.array(average_perday), axis=0)
    
#     raw_image = hv.Image(sub_array, bounds=[0,0,len(y_labels),len(x_labels)], label=mouse+'_average').redim.range(z=(-1, 1))
#     raw_image.opts(width=800, height=600, invert_axes=True, invert_yaxis=True, 
#                    invert_xaxis=True, cmap='Viridis', yticks=y_labels, xticks=x_labels,
#                    tools=['hover'], shared_axes=False, xrotation=45, aspect='equal')
#     holo_dict[(mouse, 'average')] = raw_image
    
# holo_image = hv.Layout(plot_list, group='images').cols(1)
holo_image = pn.panel(hv.HoloMap(holo_dict, kdims=['mouse', 'day']), center=True, widget_location='top')

# holo_image = hv.GridSpace(holo_dict, kdims=['mouse', 'day']).opts(plot_size=300)
holo_image


In [ ]:
# Plot the average cell activity
# # allocate a dict for the holomap
# holo_dict_a = {}
# holo_dict_b = {}
# holo_dict = {}

# # check if dictionary or dataframe
# if isinstance(data, dict):
#     # get the dates for this mouse
#     mice = data.keys()
# else:
#     mice = ['All']
# # for all the mice
# for mouse in mice:
# #     sub_dict = {}
#     # check if dictionary or dataframe
#     if isinstance(data, dict):
#         # get the dates for this mouse
#         dates = data[mouse].keys()
#     else:
#         dates = ['All']
#     # for all the dates
#     for day in dates:
#         # get the table
#         if isinstance(data, dict):
#             sub_data = data[mouse][day]
#         else:
#             sub_data = data
#         # get the available columns
#         labels = list(sub_data.columns)
#         cells = [el for el in labels if 'cell' in el]
#         not_cells = [el for el in labels if ('cell' not in el) and (el not in ['time_vector', 'frame'])]
        
        
#         # get the non-neuronal data
#         non_neural = sub_data.loc[:, not_cells]
#         labels = non_neural.columns
#         y_labels = [(idx+0.5, el) for idx, el in enumerate(labels)]
#         non_neural = fm.normalize_matrix(non_neural.to_numpy(), axis=0)
#         # get the neuronal responses
#         sub_data = sub_data.loc[:, cells]

#         # get the array
#         sub_array = np.corrcoef(sub_data.to_numpy())

#         # create the labels
#         y_labels = [(idx+0.5, el) for idx, el in enumerate(labels)]
#         raw_image = hv.Image(sub_array, bounds=[0,0,sub_array.shape[1],
#                                                 sub_array.shape[1]], label=mouse+'_'+day).redim.range(z=(-1, 1))
#         raw_image.opts(width=800, invert_axes=True, invert_yaxis=True,
#                        invert_xaxis=True, cmap='Viridis', tools=['hover'], axiswise=True)
#         # also plot the non neural data
#         param_image = hv.Image(non_neural, bounds=[0,0,len(labels),sub_array.shape[1]], label=mouse+'_'+day)
#         param_image.opts(width=800, height=200, invert_axes=True, invert_yaxis=True,
#                        invert_xaxis=True, cmap='Viridis', tools=[], yticks=y_labels, axiswise=True)
# #         # link the axes
# #         dlink = DataLink(raw_image, param_image)
        
#         # assemble the dictionary in place
# #         holo_dict_a[(mouse,day)] = hv.HoloMap({(mouse,day): raw_image}, kdims=['mouse', 'day'])
# #         holo_dict_b[(mouse,day)] = hv.HoloMap({(mouse,day): param_image}, kdims=['mouse', 'day'])
#         holo_dict_a[(mouse,day)] = raw_image
#         holo_dict_b[(mouse,day)] = param_image
# #         holo_dict[(mouse, day)] = hv.NdLayout({'corr':raw_image, 'param':param_image}, kdims='Type').cols(1)
# #         holo_dict[(mouse, day)] = param_image

# map_a = hv.HoloMap(holo_dict_a, kdims=['mouse', 'day'])
# map_b = hv.HoloMap(holo_dict_b, kdims=['mouse', 'day'])

# holo_dict = {'Correlation': map_a, 'Parameters': map_b}

# # holo_image = pn.panel(hv.HoloMap(holo_dict, kdims=['mouse', 'day']).collate(), center=True, widget_location='top')
# # create the ndlayout
# nd_image = hv.NdLayout(holo_dict, kdims=['Type']).cols(1)
# holo_image = pn.panel(nd_image, center=True, widget_location='top')

# print(nd_image)
# holo_image



In [ ]:
# Establish the relationship between neural activity and mouse speed

# sort the mouse speed in a histogram

# define the number of bins
bin_number = 20
# allocate an array 
# allocate a dict for the holomap
holo_dict = {}

# check if dictionary or dataframe
if isinstance(data, dict):
    # get the dates for this mouse
    mice = data.keys()
else:
    mice = ['All']
# for all the mice
for mouse in mice:
    sub_dict = {}
    # check if dictionary or dataframe
    if isinstance(data, dict):
        # get the dates for this mouse
        dates = data[mouse].keys()
    else:
        dates = ['All']
    # for all the dates
    for day in dates:
        # get the table
        if isinstance(data, dict):
            sub_data = data[mouse][day]
        else:
            sub_data = data
        # get the available columns
        labels = list(sub_data.columns)
        cells = [el for el in labels if 'cell' in el]
        not_cells = [el for el in labels if 'cell' not in el]
#         # get only the neural activity and the speed
#         sub_data = sub_data.loc[:, ['mouse_speed']+cells]
        # turn the calcium activity to an array
        calcium_data = sub_data.loc[:, cells].to_numpy()

        # get the histogram indexes of the cell activity
        _, bin_edges = np.histogram(sub_data['mouse_speed'],bin_number)
        bin_indexes = np.digitize(sub_data['mouse_speed'], bin_edges)
        
        # allocate an array for the activities
        activity = np.zeros((bin_number, 2))
#         print(bin_indexes)
        # for all the bins
        for bins in np.arange(bin_number):
            activity[bins, 0] = bins
            activity[bins, 1] = np.mean(calcium_data[bin_indexes==bins])
        
        # create a plot and store
        holo_dict[(mouse, day)] = hv.Bars(activity)

# # also add the compiled version
# holo_dict[('All','All')] = np.sum()

            
holo_image = pn.panel(hv.HoloMap(holo_dict, kdims=['mouse', 'day']), center=True, widget_location='top')

# holo_image = hv.GridSpace(holo_dict, kdims=['mouse', 'day']).opts(plot_size=300)
holo_image

In [ ]:
# Detect running onsets and align to neural activity

# allocate a dict for the holomap
holo_dict = {}
# allocate memory to store the images
average_speed = {}

# set the running threshold
running_threshold = 200
# define the span of the window (in s)
running_window = 2

# check if dictionary or dataframe
if isinstance(data, dict):
    # get the dates for this mouse
    mice = data.keys()
else:
    mice = ['All']
# for all the mice
for mouse in mice:
    sub_dict = {}
    # check if dictionary or dataframe
    if isinstance(data, dict):
        # get the dates for this mouse
        dates = data[mouse].keys()
    else:
        dates = ['All']
    # for all the dates
    for day in dates:
        # get the table
        if isinstance(data, dict):
            sub_data = data[mouse][day]
        else:
            sub_data = data
        # get the available columns
        labels = list(sub_data.columns)
        cells = [el for el in labels if 'cell' in el]
        not_cells = [el for el in labels if 'cell' not in el]
        
        # define the thresholding function
        def thres_function(param, thres):
            return param > thres
        
        # get the trial vector
        trial_vector = sub_data.trial_id.to_numpy()
        # get the number of trials 
        num_trials = np.unique(trial_vector).shape[0]
        # allocate memory to store the events
        event_list = []
        # for all the trials
        for trials in np.arange(num_trials):            
            # get the windows epochs
            event_list.append(fp.timed_event_finder(sub_data.iloc[trial_vector==trials, :],
                                           'mouse_speed', running_threshold, thres_function, window=2.5))
            
        if len(event_list) == 0:
            continue
        
        # concatenate the list
        onsets = pd.concat(event_list, axis=0)
        # get only the neural activity and the speed
        onsets = onsets.loc[:, ['trial_id', 'event_id']+cells]
        
        # group the events together
        onsets = onsets.groupby(['trial_id', 'event_id']).agg(list)
        # get the available columns
        labels = list(onsets.columns)
        # turn into a matrix for plotting
        onsets = np.array([el for sublist in onsets.to_numpy().T for el in sublist]).T
        
        # store for averaging
        average_speed[(mouse,day)] = onsets.T
        # normalize per row
        onsets = fm.normalize_matrix(onsets, axis=0)

        # create the labels
        y_labels = [(idx+0.5, el) for idx, el in enumerate(labels)]

        raw_image = hv.Image(onsets, bounds=[0,0,len(y_labels),10], label=mouse+'_'+day)
        raw_image.opts(width=800, height=600, invert_axes=True, invert_yaxis=True, 
                       invert_xaxis=True, cmap='Viridis', yticks=y_labels, tools=['hover'], shared_axes=False)
        holo_dict[(mouse,day)] = raw_image
        

holo_image = pn.panel(hv.HoloMap(holo_dict, kdims=['mouse', 'day']), center=True, widget_location='top')

holo_image

In [ ]:
# Plot the averaging
# allocate a dict for the holomap
holo_dict = {}

# for every mouse and day

# check if dictionary or dataframe
if isinstance(data, dict):
    # get the dates for this mouse
    mice = data.keys()
else:
    mice = ['All']
# for all the mice
for mouse in mice:
    sub_dict = {}
    # check if dictionary or dataframe
    if isinstance(data, dict):
        # get the dates for this mouse
        dates = data[mouse].keys()
    else:
        dates = ['All']
    # for all the dates
    for day in dates:
        # get the table
        if isinstance(data, dict):
            sub_data = data[mouse][day]
        else:
            sub_data = data
        # get the data
        plot_data = average_speed[(mouse, day)]
        # get the number of points
        x_range = np.arange(plot_data.shape[1])
        # get the average and std
        data_mean = np.mean(plot_data, axis=0)
        data_sem = np.std(plot_data, axis=0)/np.sqrt(plot_data.shape[0])
        overlay_plot = hv.Curve((x_range, data_mean))*hv.Spread((x_range, data_mean, data_sem))
        holo_dict[(mouse,day)] = overlay_plot
        
hmap = hv.HoloMap(holo_dict, kdims=['mouse', 'day'])
holo_image = pn.panel(hmap, center=True, widget_location='top')

holo_image